In [1]:
%load_ext autoreload
%autoreload 2

import csv
import ipywidgets as widgets
import numpy as np
import os
import pandas as pd
import time

from benchmarks import gsa_svm_fitness
from src.entities import GSA

from IPython.display import display
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from typing import Mapping, Tuple, Union
from ucimlrepo import fetch_ucirepo

In [2]:
uci_datasets = ["(Small) Breast Cancer Wisconsin (Diagnostic)",
                "(Mid) Spambase",
                "(Large) Mushroom"]

widget_opt = widgets.Dropdown(
    options=uci_datasets,
    description='Dataset: '
)

display(widget_opt)



Dropdown(description='Dataset: ', options=('(Small) Breast Cancer Wisconsin (Diagnostic)', '(Mid) Spambase', '…

In [3]:
def fetch_categorical_dataset(_id: int) -> Tuple[pd.DataFrame, pd.Series]:
    data = fetch_ucirepo(id=_id)
    X = data.data.features
    y = data.data.targets
    encoder = LabelEncoder()
    for col in X.columns:
        X.loc[:, col] = encoder.fit_transform(X[col])
    return X, y

widget_opt.value = "(Mid) Spambase"

if widget_opt.value == "(Small) Breast Cancer Wisconsin (Diagnostic)":
    data = fetch_ucirepo(id=15)
    X = data.data.features
    y = data.data.targets
    X = X.fillna(value=0)
elif widget_opt.value == "(Mid) Spambase":
    X, y = fetch_categorical_dataset(_id=94)
elif widget_opt.value == "(Large) Mushroom":
    X, y = fetch_categorical_dataset(_id=73)
    
# Name of selected dataset and summary (number of instances, features, etc.)
print(f"Dataset: {widget_opt.value}")
print(f"Instances: {X.shape[0]} (Train / Test: 80% / 20%)")
print(f"Features: {X.shape[1]}")

Dataset: (Mid) Spambase
Instances: 4601 (Train / Test: 80% / 20%)
Features: 57


In [4]:
class UCI:
    """
    Class to handle UCI datasets
    
    Args:
        X (pd.DataFrame): Features
        y (pd.Series): Target
    """
    def __init__(self,
                 X: pd.DataFrame,
                 y: pd.Series,
                 boundaries: Mapping[str, Tuple[Tuple[float, float], ...]]
                 ) -> None:
        """
        Constructor
        
        Args:
            X (pd.DataFrame): Features
            y (pd.Series): Target
        """
        self.X = X
        self.y = y
        self.boundaries = boundaries

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.X)
        
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.20, random_state=5)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

    
    def get_fitness(self,
                    solution: Mapping[str, np.ndarray],
                    data: Union[None, Tuple[np.ndarray, np.ndarray]] = None,
                    ) -> Tuple[float, float]:
        """
        Get fitness of a solution
        
        Args:
            solution (Mapping[str, np.ndarray]): Solution to evaluate
            data (Union[None, Tuple[np.ndarray, np.ndarray]], optional): Data to evaluate the solution. Defaults to None.
        
        Returns:
            Tuple[float, float]: Fitness and accuracy of the solution    
        """
        if data is None:
            X_scaled = self.X_train
            y_data = self.y_train
        else:
            X_scaled, y_data = data
            
        gamma, C = solution['real']
        gamma /= 1000
        C /= 1000
        X_scaled_filtered = X_scaled[:, solution['discrete'].astype(int) == 1]
        svc_model = SVC(gamma=gamma, C=C, kernel="rbf", verbose=False)
        svc_model.fit(X_scaled_filtered, np.ravel(y_data))
        y_predict = svc_model.predict(X_scaled_filtered)
        conf_matrix = confusion_matrix(y_data, y_predict)
        accuracy = accuracy_score(y_data, y_predict) * 100
        
        return gsa_svm_fitness(accuracy=accuracy, solution=solution)

    def is_feasible(self, solution: Mapping[str, np.ndarray]) -> bool:
        """
        Check if a solution is feasable

        Args:
            solution (Mapping[str, np.ndarray]): Solution to evaluate

        Returns:
            bool: True if the solution is feasable, False otherwise
        """
        real_values = solution['real']
        discrete_values = solution['discrete']

        for i, (min_val, max_val) in enumerate(self.boundaries['real']):
            if real_values[i] < min_val or real_values[i] > max_val:
                return False

        for i, (min_val, max_val) in enumerate(self.boundaries['discrete']):
            if discrete_values[i] < min_val or discrete_values[i] > max_val:
                return False

        return True

boundaries = {'real': [(1, 100_000), (1, 100_000)], 'discrete': [(0, 1) for _ in range(len(X.columns))]}
uci = UCI(X, y, boundaries)

In [12]:
# Select number of repetitions for each experiment. 
runs = 10

# Select chaotic constant
chaotic_constant = True

# Export results ?
export = True

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

atLeastOneIteration = False

chaotic_constant=True
repair_solution=True

population_size = 5 
iterations = 20

# CSV Header for the convergence
CnvgHeader = ["Sol_Iter"+str(l+1) for l in range(iterations)]

best_solution_history = []
accuracy_history = []
gamma_history = []
c_history = []
n_features_history = []

uci = UCI(X, y, boundaries)

for k in range(runs):
    gsa_algo = GSA(objective_function = uci.get_fitness,
                   is_feasible=uci.is_feasible,
                   r_dim=2,
                   d_dim=len(X.columns),
                   boundaries=uci.boundaries,
                   repair_solution=repair_solution)
    
    gsa_algo.optimize(population_size=population_size,
                      iters=iterations,
                      chaotic_constant=chaotic_constant)
    
    print(gsa_algo.solution_history[-1])
    fitness, accuracy = uci.get_fitness(solution=gsa_algo.solution_history[-1],
                                        data=(uci.X_test, uci.y_test))
    
    print("Test accuracy: ", accuracy, " - Fitness: ", fitness)
    
    best_solution_history.append(gsa_algo.solution_history[-1])
    accuracy_history.append(accuracy)
    gamma_history.append(gsa_algo.solution_history[-1]['real'][0] / 1000)
    c_history.append(gsa_algo.solution_history[-1]['real'][1] / 1000)
    n_features_history.append(np.sum(gsa_algo.solution_history[-1]['discrete']))
    
    if export:
        with open(ExportToFile, 'a') as out:
            writer = csv.writer(out, delimiter=',')
            if not atLeastOneIteration:  # just one time to write the header of the CSV file
                header = np.concatenate(
                    [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                writer.writerow(header)
            a = np.concatenate(
                [[gsa_algo.objective_function_name,
                  gsa_algo.start_time,
                  gsa_algo.end_time,
                  gsa_algo.execution_time],
                  gsa_algo.convergence])
            writer.writerow(a)
        out.close()
    atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Failed to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "get_fitness"
['At iteration 1 the best fitness is 92.32723112128147']
['At iteration 2 the best fitness is 92.32723112128147']
['At iteration 3 the best fitness is 92.32723112128147']
['At iteration 4 the best fitness is 92.32723112128147']
['At iteration 5 the best fitness is 92.32723112128147']
['At iteration 6 the best fitness is 92.32723112128147']
['At iteration 7 the best fitness is 92.32723112128147']
['At iteration 8 the best fitness is 92.32723112128147']
['At iteration 9 the best fitness is 92.32723112128147']
['At iteration 10 the best fitness is 92.32723112128147']
['At iteration 11 the best fitness is 92.32723112128147']
['At iteration 12 the best fitness is 92.32723112128147']
['At iteration 13 the best fitness is 92.32723112128147']
['At iteration 14 the best fitness is 92.32723112128147']
['At iteration 15 the best fitness is 92.32723112128147']
['At iteration 16 the best fitness is 92.32723112128147']
['At iteration 17 the best fitness is 92.3272311

In [13]:
best_solution_history

[{'real': array([33351.098612  , 61636.53556799]),
  'discrete': array([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
         0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0])},
 {'real': array([33372.31087857, 22767.85079467]),
  'discrete': array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
         1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1])},
 {'real': array([91730.12581458, 27842.14352057]),
  'discrete': array([0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
         0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0])},
 {'real': array([89603.61577872,  3599.08386987]),
  'discrete': array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0

In [14]:
accuracy_history

[99.2399565689468,
 99.45711183496199,
 98.37133550488599,
 99.1313789359392,
 99.6742671009772,
 99.8914223669924,
 99.8914223669924,
 100.0,
 99.45711183496199,
 99.6742671009772]

In [15]:
gamma_history

[33.35109861199627,
 33.37231087857161,
 91.73012581457714,
 89.60361577871588,
 77.65883242259503,
 88.8767369631541,
 58.880851297745565,
 91.31480523795013,
 5.761769615250105,
 31.78355948531779]

In [16]:
c_history

[61.636535567994336,
 22.767850794673954,
 27.842143520572588,
 3.5990838698712624,
 91.25770820986376,
 19.509393992523034,
 12.442035655760169,
 56.12363724019844,
 27.23340512769299,
 50.591802173815935]

In [17]:
mean_accuracy = np.mean(accuracy_history)
std_accuracy = np.std(accuracy_history)

print(f"Mean accuracy: {mean_accuracy} +/- {std_accuracy}")

mean_gamma = np.mean(gamma_history)
std_gamma = np.std(gamma_history)

print(f"Mean gamma: {mean_gamma} +/- {std_gamma}")

mean_c = np.mean(c_history)
std_c = np.std(c_history)

print(f"Mean C: {mean_c} +/- {std_c}")

mean_n_features = np.mean(n_features_history)
std_n_features = np.std(n_features_history)

print(f"Mean n_features: {mean_n_features} +/- {std_n_features}")

Mean accuracy: 99.47882736156352 +/- 0.4575745386689587
Mean gamma: 60.23337061058736 +/- 30.277633787605748
Mean C: 37.30035961529664 +/- 25.50852623123389
Mean n_features: 26.7 +/- 4.1


In [18]:
# Save results to csv with following columns:
# dataset_name, r_runs, n_pop n_iters, chaotic_constant, repair_solution, mean_accuracy, std_accuracy, mean_gamma, std_gamma, mean_c, std_c, mean_n_features, std_n_features
results = pd.DataFrame({"dataset_name": [widget_opt.value],
                        "r_runs": [runs],
                        "n_pop": [population_size],
                        "n_iters": [iterations],
                        "chaotic_constant": [chaotic_constant],
                        "repair_solution": [repair_solution],
                        "mean_accuracy": [mean_accuracy],
                        "std_accuracy": [std_accuracy],
                        "mean_gamma": [mean_gamma],
                        "std_gamma": [std_gamma],
                        "mean_c": [mean_c],
                        "std_c": [std_c],
                        "mean_n_features": [mean_n_features],
                        "std_n_features": [std_n_features]})

if os.path.exists("data/gsa_records.csv"):
    results.to_csv("data/gsa_records.csv", mode="a", header=False, index=False)
else: # Append row to existing file
    results.to_csv("data/gsa_records.csv", index=False)

                /\ 
               /  \
                || 
                
        Nuestros resultados
        
Dataset: Mushroom
Instances: 8124 (Train / Test: 80% / 20%)
Features: 22

Resultados autores:

| Metric | Value             |
| --- |-------------------|
| Accuracy | 98.06 +/- 0.78    |
| Gamma | 0.0067 +/- 0.0144 |
| C | 47.35 +/- 27.57   |
| n_features | 3 +/- 2.16        |

                /\ 
               /  \
                || 
                
        Nuestros resultados
        
Dataset: Breast Cancer Wisconsin (Diagnostic) Data Set

Resultados autores:

| Metric | Value             |
| --- |-------------------|
| Accuracy | 99.54 +/- 0.25    |
| Gamma | 0.0685 +/- 0.1293 |
| C | 40.30 +/- 22.37   |
| n_features | 2 +/- 1           |